# सत्र २ – RAG मूल्यांकन रागाससँग

रागास मेट्रिक्स प्रयोग गरेर न्यूनतम RAG पाइपलाइनको मूल्यांकन गर्नुहोस्: उत्तर सान्दर्भिकता, विश्वसनीयता, सन्दर्भ सटीकता।


# परिदृश्य
यो परिदृश्यले न्यूनतम Retrieval Augmented Generation (RAG) पाइपलाइनलाई स्थानीय रूपमा मूल्याङ्कन गर्दछ। हामी:
- सानो कृत्रिम कागजात संग्रह परिभाषित गर्छौं।
- कागजातहरूलाई एम्बेड गर्छौं र साधारण समानता खोजी गर्ने प्रणाली कार्यान्वयन गर्छौं।
- स्थानीय मोडेल (Foundry Local / OpenAI-संगत) प्रयोग गरेर आधारभूत उत्तरहरू उत्पन्न गर्छौं।
- ragas मेट्रिक्स (`answer_relevancy`, `faithfulness`, `context_precision`) गणना गर्छौं।
- छिटो पुनरावृत्तिका लागि FAST मोड (env `RAG_FAST=1`) लाई समर्थन गर्छौं, जसले केवल उत्तरको सान्दर्भिकता गणना गर्छ।

यो नोटबुक प्रयोग गरेर सुनिश्चित गर्नुहोस् कि तपाईंको स्थानीय मोडेल + एम्बेडिङ स्ट्याकले ठूला कागजात संग्रहमा विस्तार गर्नु अघि तथ्यमा आधारित उत्तरहरू उत्पादन गर्छ।


### व्याख्या: निर्भरता स्थापना
आवश्यक पुस्तकालयहरू स्थापना गर्दछ:
- स्थानीय मोडेल व्यवस्थापनको लागि `foundry-local-sdk`।
- `openai` क्लाइन्ट इन्टरफेसको लागि।
- घना एम्बेडिङहरूको लागि `sentence-transformers`।
- मूल्याङ्कन र मेट्रिक गणनाको लागि `ragas` + `datasets`।
- ragas LLM इन्टरफेसको लागि `langchain-openai` एडाप्टर।

पुन: चलाउन सुरक्षित; यदि वातावरण पहिले नै तयार छ भने छोड्न सकिन्छ।


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### व्याख्या: कोर आयातहरू र मेट्रिक्स
कोर पुस्तकालयहरू र रागा मेट्रिक्स लोड गर्दछ। मुख्य अंशहरू:
- एम्बेडिङको लागि SentenceTransformer।
- `evaluate` + चयन गरिएका रागा मेट्रिक्स।
- मूल्याङ्कन कोष निर्माण गर्न `Dataset`।
यी आयातहरूले टाढाको कलहरू ट्रिगर गर्दैनन् (एम्बेडिङको लागि सम्भावित मोडेल क्यास लोड बाहेक)।


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### व्याख्या: टॉय कर्पस र QA ग्राउन्ड ट्रुथ
एक सानो इन-मेमोरी कर्पस (`DOCS`), प्रयोगकर्ताका प्रश्नहरूको सेट, र अपेक्षित ग्राउन्ड ट्रुथ उत्तरहरू परिभाषित गर्दछ। यीले बाह्य डेटा फेच नगरी छिटो र निर्धारणात्मक मेट्रिक गणना गर्न अनुमति दिन्छ। वास्तविक परिदृश्यहरूमा, तपाईं उत्पादन क्वेरीहरू + क्युरेटेड उत्तरहरू नमूना गर्नुहुनेछ।


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### व्याख्या: सेवा आरम्भ, एम्बेडिङ्स र सुरक्षा प्याच
Foundry Local प्रबन्धकलाई आरम्भ गर्छ, `promptTemplate` को लागि स्किमा-ड्रिफ्ट सुरक्षा प्याच लागू गर्छ, मोडेल आईडी समाधान गर्छ, OpenAI-संगत क्लाइन्ट सिर्जना गर्छ, र कागजात संग्रहको लागि घना एम्बेडिङ्स पूर्व-गणना गर्छ। यसले पुन: प्रयोग गर्न मिल्ने अवस्था पुनःप्राप्ति + उत्पादनको लागि तयार गर्छ।


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### व्याख्या: Retriever Function
सामान्यीकृत embeddings मा डट प्रोडक्ट प्रयोग गरेर साधारण भेक्टर समानता retriever परिभाषित गर्दछ। शीर्ष-k डकहरू (k=2 डिफल्ट) फिर्ता गर्छ। उत्पादनमा स्केल र लेटेंसीका लागि ANN इन्डेक्स (FAISS, Chroma, Milvus) सँग परिवर्तन गर्नुहोस्।


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### व्याख्या: जेनेरेशन फङ्सन
`generate` ले सीमित प्रम्प्ट निर्माण गर्छ (सिस्टमले मात्र सन्दर्भ प्रयोग गर्न निर्देशन दिन्छ) र स्थानीय मोडेललाई कल गर्छ। कम तापक्रम (0.1) ले सिर्जनशीलताभन्दा सत्यतामा आधारित निष्कर्षलाई प्राथमिकता दिन्छ। काटिएको उत्तर पाठ फर्काउँछ।


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### व्याख्या: फलब्याक क्लाइन्ट इनिसियलाइजेसन
`client` को अस्तित्व सुनिश्चित गर्दछ, चाहे पहिलेको इनिसियलाइजेसन सेल छुटेको होस् वा असफल भएको हो—पछिल्ला मूल्याङ्कन चरणहरूमा NameError हुनबाट जोगाउँछ।


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### व्याख्या: मूल्यांकन लूप र मेट्रिक्स
मूल्यांकन डेटासेट तयार गर्दछ (आवश्यक स्तम्भहरू: प्रश्न, उत्तर, सन्दर्भहरू, वास्तविक उत्तरहरू, सन्दर्भ) र त्यसपछि चयन गरिएका रागास मेट्रिक्सहरूमा पुनरावृत्ति गर्दछ।

अप्टिमाइजेसन:
- FAST_MODE ले छिटो परीक्षणका लागि उत्तरको सान्दर्भिकतालाई मात्र सीमित गर्दछ।
- प्रति-मेट्रिक लूपले कुनै एक मेट्रिक असफल हुँदा पूर्ण पुन:गणना हुनबाट जोगाउँछ।

एक dict उत्पादन गर्दछ जसमा मेट्रिक -> स्कोर हुन्छ (असफल भएमा NaN)।


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
